In [1]:
import sys
import math
sys.path.append("..")

import torch

from algorithms.deepinversion import distill_deep_inversion
from algorithms.vanilla_distillation import distill
# LATER CHANGE TO SINGLE FUNCTION WITH METHODS FROM CONFIG!

from trainer import Trainer
from conv_net import SimpleConvNet
from utils import get_cifar100_loader

device = "cuda"

%load_ext autoreload
%autoreload 2

In [2]:
teacher = SimpleConvNet(n_layers=10, n_filters=64, input_shape=(3, 32, 32), n_classes=100).to(device)
student_reference = SimpleConvNet(n_layers=2, n_filters=16, input_shape=(3, 32, 32), n_classes=100).to(device)

student_vanilla = SimpleConvNet(n_layers=2, n_filters=16, input_shape=(3, 32, 32), n_classes=100).to(device)
student_deepinversion = SimpleConvNet(n_layers=2, n_filters=16, input_shape=(3, 32, 32), n_classes=100).to(device)

In [3]:
train_config = {
    'lr': 1e-3,
    'batch_size': 512,
    'max_iters': -1,
    'max_epochs': 7,
    'evaluate_every': 36,
    'log_every': 36,
    "optimizer": "AdamW",
    'verbose': False,
    'dataset_size': 50000
}

In [4]:
trainer = Trainer(train_config=train_config)

In [5]:
teacher = trainer.train(
    model=teacher,
    train_loader=get_cifar100_loader(data_path="../../data", train=True, batch_size=train_config["batch_size"]),
    val_loader=get_cifar100_loader(data_path="../../data", train=False, batch_size=train_config["batch_size"])
)

Training iters:   0%|          | 0/350000 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

...

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
student_reference = trainer.train(
    model=student_reference,
    train_loader=get_cifar100_loader(data_path="../../data", train=True, batch_size=train_config["batch_size"]),
    val_loader=get_cifar100_loader(data_path="../../data", train=False, batch_size=train_config["batch_size"])
)

Training iters:   0%|          | 0/350000 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

...

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

eval batch:   0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
loader_temp = get_cifar100_loader(data_path="../../data", train=True, batch_size=train_config["batch_size"])

In [8]:
student_vanilla = distill(
    teacher=teacher,
    student=student_vanilla,
    optimizer=torch.optim.AdamW(student_vanilla.parameters(), lr=train_config["lr"], betas=(0.8, 0.9)),
    train_loader=get_cifar100_loader(data_path="../../data", train=True, batch_size=train_config["batch_size"]),
    test_loader=None,
    iterations=math.ceil(train_config['dataset_size'] / train_config["batch_size"]) * train_config["max_epochs"],
    test_freq=-1,
    alpha=0.6,
    T=2.5
)

Training - 0/686 [                              ] ELP: 00:00d:\Desktop\10сем\BMM\DFDistill\.venv\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Training - 686/686 [██████████████████████████████] ELP: 00:49, accuracy: 0.2772 - 


In [ ]:
student_deepinversion = distill_deep_inversion(
    teacher=teacher.to("cpu"), 
    student=student_deepinversion.to("cpu"),
    distill_config={
        "alpha": 0.6,
        "T": 2.5,
        "lr": 0.001
    },
    total_iterations=100000,
    distill_k_times=8,
    batch_size=256,
    deep_inversion_batch_size=1024,
    deep_inversion_iterations=200,
    n_classes=100,
    image_shape=(3, 32, 32),
    r_feature=0.01,
    first_bn_multiplier=10,
    tv_l1=0.0,
    tv_l2=0.0001,
    l2=0.00001,
    main_loss_multiplier=1.0,
    lr=0.1,
    adi_scale=1,
    device="cpu"
)

Pipeline iters:   0%|          | 0/97 [00:00<?, ?it/s]

Getting_images:   0%|          | 0/200 [00:00<?, ?it/s]

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor